# Polity5

In [18]:
import pandas as pd
df = pd.read_excel('../datasets/raw/democracy/polity/POLITY5-PRC.xlsx', sheet_name='Data')

# Rename columns
df.rename(columns={'Economy ISO3': 'country_iso', 'Economy Name': 'country_name'}, inplace=True)

df['Indicator ID'] = df['Indicator ID'].str.replace(r'^POLITY5\.PRC\.', '', regex=True)


# Drop the columns Attribute 1	Attribute 2	Attribute 3
df.drop(columns=['Attribute 1', 'Attribute 2', 'Attribute 3'], inplace=True)

# Set the years as identifiers instead of columns
years = [str(year) for year in range(1776, 2019)]
df_long = df.melt(
    id_vars=['country_iso', 'country_name', 'Indicator ID'],
    value_vars=years,
    var_name='Year',
    value_name='Value'
)

df_long['Year'] = pd.to_numeric(df_long['Year'])

# Set the Indicator as the column
df_final = df_long.pivot_table(
    index=['country_iso', 'country_name', 'Year'],
    columns='Indicator ID',
    values='Value'
).reset_index()

df_final.to_csv('../datasets/processed/democracy/polity/polity5.csv', index=False)

df_final.head()

Indicator ID,country_iso,country_name,Year,autoc,democ,durable,exconst,exrec,parcomp,parreg,polcomp,polity,polity2,xconst,xrcomp,xropen,xrreg
0,AFG,Afghanistan,1800,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
1,AFG,Afghanistan,1801,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
2,AFG,Afghanistan,1802,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
3,AFG,Afghanistan,1803,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0
4,AFG,Afghanistan,1804,7.0,1.0,NaN,1.0,1.0,3.0,3.0,6.0,-6.0,-6.0,1.0,1.0,1.0,3.0


In [19]:
countries = df_final["country_name"].unique()

print(f"Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")


Nr of countries: 166 - Countries: Afghanistan, Angola, Albania, United Arab Emirates, Argentina, Armenia, Australia, Austria, Azerbaijan, Burundi, Belgium, Benin, Burkina Faso, Bangladesh, Bulgaria, Bahrain, Bosnia and Herzegovina, Belarus, Bolivia, Brazil, Bhutan, Botswana, Central African Republic, Canada, Switzerland, Chile, China, Cote d'Ivoire, Cameroon, Congo, Rep., Colombia, Comoros, Cabo Verde, Costa Rica, Cuba, Cyprus, Czechia, Germany, Djibouti, Denmark, Dominican Republic, Algeria, Ecuador, Egypt, Arab Rep., Eritrea, Spain, Estonia, Ethiopia, Finland, Fiji, France, Gabon, United Kingdom, Georgia, Ghana, Guinea, Gambia, The, Guinea-Bissau, Equatorial Guinea, Greece, Guatemala, Guyana, Honduras, Croatia, Haiti, Hungary, Indonesia, India, Ireland, Iran, Islamic Rep., Iraq, Israel, Italy, Jamaica, Jordan, Japan, Kazakhstan, Kenya, Kyrgyz Republic, Cambodia, Korea, Rep., Kuwait, Lao PDR, Lebanon, Liberia, Libya, Sri Lanka, Lesotho, Lithuania, Luxembourg, Latvia, Morocco, Moldova,

# Freedom in the World

In [25]:
import pandas as pd

df = pd.read_excel('../datasets/raw/democracy/freedom-world/All_data_FIW_2013-2024.xlsx', sheet_name='FIW13-25', header=1)

df.rename(columns={'Country/Territory': 'country', 'Edition': 'year'}, inplace=True)
df.drop(columns=['Region'], inplace=True)
df = df[df['C/T'] == 'c']
df.drop(columns=['Add Q', 'Add A', 'C/T'], inplace=True)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.to_csv('../datasets/processed/democracy/freedom-world/freedom-world.csv', index=False)
df.head()

,country,year,status,pr_rating,cl_rating,a1,a2,a3,a,b1,...,f3,f4,f,g1,g2,g3,g4,g,cl,total
1,Afghanistan,2025,NF,7,7,0,0,0,0,0,...,0,0,0,0,1,0,1,2,5,6
2,Albania,2025,PF,3,3,3,3,3,9,3,...,2,3,10,3,2,2,2,9,40,68
3,Algeria,2025,NF,6,5,1,1,1,3,1,...,2,2,6,2,2,2,1,7,21,31
4,Andorra,2025,F,1,1,4,4,4,12,4,...,4,3,15,4,4,3,4,15,55,93
5,Angola,2025,NF,6,5,0,2,1,3,1,...,1,2,5,1,1,1,0,3,18,28


In [26]:
countries = df["country"].unique()

print(f"Nr of countries: {len(countries)} - Countries: {', '.join(countries)}")

print(f"From years {df['year'].min()} to {df['year'].max()}")


Nr of countries: 195 - Countries: Afghanistan, Albania, Algeria, Andorra, Angola, Antigua and Barbuda, Argentina, Armenia, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belarus, Belgium, Belize, Benin, Bhutan, Bolivia, Bosnia and Herzegovina, Botswana, Brazil, Brunei, Bulgaria, Burkina Faso, Burundi, Cabo Verde, Cambodia, Cameroon, Canada, Central African Republic, Chad, Chile, China, Colombia, Comoros, Congo (Brazzaville), Congo (Kinshasa), Costa Rica, Cote d'Ivoire, Croatia, Cuba, Cyprus, Czech Republic, Denmark, Djibouti, Dominica, Dominican Republic, Ecuador, Egypt, El Salvador, Equatorial Guinea, Eritrea, Estonia, Eswatini, Ethiopia, Fiji, Finland, France, Gabon, Georgia, Germany, Ghana, Greece, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Haiti, Honduras, Hungary, Iceland, India, Indonesia, Iran, Iraq, Ireland, Israel, Italy, Jamaica, Japan, Jordan, Kazakhstan, Kenya, Kiribati, Kosovo, Kuwait, Kyrgyzstan, Laos, Latvia, Lebanon, Lesotho, Liberia, Li